In [0]:
!pip install soundfile
!pip install librosa

In [0]:
from google.colab import drive
#import soundfile
import librosa
import pickle
from sklearn.svm import SVC 
from sklearn.naive_bayes import GaussianNB
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [70]:
#создаю датафрейм для аудио
data = pd.DataFrame (columns=['Name_audio','Audio', 'Feature_Tone', 'Tone.Ravel'])
data

,Name_audio,Audio,Feature_Tone,Tone.Ravel


In [71]:
#получаю список названий файлов в папке с аудио
path = "/content/gdrive/My Drive/audio_train/train/"
name_files = os.listdir(path)
data['Name_audio'] = name_files #записываю названия в датафрейм
data

,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,NaN,NaN,NaN
1,f9b80a8a5903c7006f7a.wav,NaN,NaN,NaN
2,70b9edbb2fb73de11b45.wav,NaN,NaN,NaN
3,f98ff11b6ab64c31ac6d.wav,NaN,NaN,NaN
4,f98de3d40918553d4599.wav,NaN,NaN,NaN
...,...,...,...,...
5678,255da5ed7dafcb5f2e64.wav,NaN,NaN,NaN
5679,0fa7cca006b8c6a9b969.wav,NaN,NaN,NaN
5680,252b858b428897138f84.wav,NaN,NaN,NaN
5681,250b6832901c953a672d.wav,NaN,NaN,NaN


In [0]:
#прохожусь по всем аудио и в датафрейм записываю полученные значения ?амплитуд? с помощью librosa
for i in range(len(name_files)):
  data['Audio'][i], fs = librosa.load(path + name_files[i] )
  #if i%500 == 0:
  print(i)
data.head()

In [0]:
#сохраняю значение амплитуд в формате pickle
with open('/content/gdrive/My Drive/Colab Notebooks/audio.pickle', 'wb') as f:
...     pickle.dump(list(data['Audio']), f)

In [0]:
#выгружаю из pickle значения амплитуд
with open('/content/gdrive/My Drive/Colab Notebooks/audio.pickle', 'rb') as f:
...     Audio = pickle.load(f)

In [73]:
data['Audio'] = Audio
data.head()

,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...",NaN,NaN
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...",NaN,NaN
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...",NaN,NaN
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...",NaN,NaN
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...",NaN,NaN


In [0]:
#ПОЛУЧЕНИЕ ПРИЗНАКОВ
#этот раздел выполняется один раз, для построения классификатора достаточно будет выгрузить все признаки из pickle

In [0]:
#1 n_fft=2048, hop_length=512

In [0]:
#получение признаков tone.Параметры по умолчанию размер окна n_fft=2048, длина перекрытия hop_length=512
for i in range(len(name_files)):
  print(i)
  data['Feature_Tone'][i] = librosa.feature.chroma_stft(data['Audio'][i])
#data.head()

In [0]:
#сохраняю признаки в формате pickle
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone.pickle', 'wb') as f:
...     pickle.dump(list(data['Feature_Tone']), f)

In [0]:
#переформировываю масив в одномерный для построения классификаторв
for i in range(len(name_files)):
  data['Tone.Ravel'][i] = data['Feature_Tone'][i].ravel()
data.head()

In [0]:
#сохраняю признаки в формате pickle
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel.pickle', 'wb') as f:
...     pickle.dump(list(data['Tone.Ravel']), f)

In [0]:
#2 n_fft=1024 hop_length=512

In [0]:
#получение признаков tone.Параметры по умолчанию размер окна n_fft=1024, длина перекрытия hop_length=512
for i in range(len(name_files)):
  #print(i)
  data['Feature_Tone'][i] = librosa.feature.chroma_stft(data['Audio'][i], n_fft=1024, hop_length=512)
data.head()

In [0]:
#сохраняю признаки в формате pickle
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_1024_512.pickle', 'wb') as f:
...     pickle.dump(list(data['Feature_Tone']), f)

In [0]:
#переформировываю масив в одномерный для построения классификаторв
for i in range(len(name_files)):
  data['Tone.Ravel'][i] = data['Feature_Tone'][i].ravel()
#data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_1024_512.pickle', 'wb') as f:
...     pickle.dump(list(data['Tone.Ravel']), f)

In [0]:
#3 n_fft=2048 hop_length=1024

In [0]:
#получение признаков tone.Параметры по умолчанию размер окна n_fft=2048, длина перекрытия hop_length=1024
for i in range(len(name_files)):
  #print(i)
  data['Feature_Tone'][i] = librosa.feature.chroma_stft(data['Audio'][i], n_fft=2048, hop_length=1024)
data.head()

In [0]:
#сохраняю признаки в формате pickle
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_2048_1024.pickle', 'wb') as f:
...     pickle.dump(list(data['Feature_Tone']), f)

In [0]:
#переформировываю масив в одномерный для построения классификаторв
for i in range(len(name_files)):
  data['Tone.Ravel'][i] = data['Feature_Tone'][i].ravel()
#data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_2048_1024.pickle', 'wb') as f:
...     pickle.dump(list(data['Tone.Ravel']), f)

In [0]:
#4 n_fft=2048 hop_length=2048

In [0]:
#получение признаков tone.Параметры по умолчанию размер окна n_fft=2048, длина перекрытия hop_length=2048
for i in range(len(name_files)):
  #print(i)
  data['Feature_Tone'][i] = librosa.feature.chroma_stft(data['Audio'][i], n_fft=2048, hop_length=2048)
data.head()

In [0]:
#сохраняю признаки в формате pickle
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_2048_2048.pickle', 'wb') as f:
...     pickle.dump(list(data['Feature_Tone']), f)

In [0]:
#переформировываю масив в одномерный для построения классификаторв
for i in range(len(name_files)):
  data['Tone.Ravel'][i] = data['Feature_Tone'][i].ravel()
#data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_2048_2048.pickle', 'wb') as f:
...     pickle.dump(list(data['Tone.Ravel']), f)

In [0]:
#5 n_fft=2048 hop_length=4096

In [0]:
#получение признаков tone.Параметры по умолчанию размер окна n_fft=2048, длина перекрытия hop_length=4096
for i in range(len(name_files)):
  #print(i)
  data['Feature_Tone'][i] = librosa.feature.chroma_stft(data['Audio'][i], n_fft=2048, hop_length=4096)
data.head()

In [0]:
#сохраняю признаки в формате pickle
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_2048_4096.pickle', 'wb') as f:
...     pickle.dump(list(data['Feature_Tone']), f)

In [0]:
#переформировываю масив в одномерный для построения классификаторв
for i in range(len(name_files)):
  data['Tone.Ravel'][i] = data['Feature_Tone'][i].ravel()
#data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_2048_4096.pickle', 'wb') as f:
...     pickle.dump(list(data['Tone.Ravel']), f)

In [0]:
#6 n_fft=4096 hop_length=4096

In [0]:
#получение признаков tone.Параметры по умолчанию размер окна n_fft=4096, длина перекрытия hop_length=4096
for i in range(len(name_files)):
  #print(i)
  data['Feature_Tone'][i] = librosa.feature.chroma_stft(data['Audio'][i], n_fft=4096, hop_length=4096)
data.head()

In [0]:
#сохраняю признаки в формате pickle
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_4096_4096.pickle', 'wb') as f:
...     pickle.dump(list(data['Feature_Tone']), f)

In [0]:
#переформировываю масив в одномерный для построения классификаторв
for i in range(len(name_files)):
  data['Tone.Ravel'][i] = data['Feature_Tone'][i].ravel()
#data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_4096_4096.pickle', 'wb') as f:
...     pickle.dump(list(data['Tone.Ravel']), f)

In [0]:
#7 n_fft=4096 hop_length=2048

In [0]:
#получение признаков tone.Параметры по умолчанию размер окна n_fft=4096, длина перекрытия hop_length=2048
for i in range(len(name_files)):
  #print(i)
  data['Feature_Tone'][i] = librosa.feature.chroma_stft(data['Audio'][i], n_fft=4096, hop_length=2048)
data.head()

In [0]:
#сохраняю признаки в формате pickle
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_4096_2048.pickle', 'wb') as f:
...     pickle.dump(list(data['Feature_Tone']), f)

In [0]:
#переформировываю масив в одномерный для построения классификаторв
for i in range(len(name_files)):
  data['Tone.Ravel'][i] = data['Feature_Tone'][i].ravel()
#data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_4096_2048.pickle', 'wb') as f:
...     pickle.dump(list(data['Tone.Ravel']), f)

In [0]:
#ПОСТРОЕНИЕ КЛАССИФИКАТОРА

In [0]:
#1 n_fft=2048, hop_length=512

In [0]:
#выгружаю из pickle признаки tone
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone.pickle', 'rb') as f:
...     Feature_Tone = pickle.load(f)

In [0]:
data['Feature_Tone'] = Feature_Tone
data.head()

,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,NaN,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.028820622776791868, 0.035540443717563326, ...",NaN
1,NaN,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.43014247714434317, 0.21300254030154597, 0....",NaN
2,NaN,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.26519665180603347, 0.2001198163939377, 0.1...",NaN
3,NaN,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.003857842635125953, 0.0006431823830839563,...",NaN
4,NaN,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.12975261057595686, 0.3552813475665379, 0.0...",NaN


In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel.pickle', 'rb') as f:
...     Tone_Ravel = pickle.load(f)

In [0]:
data['Tone.Ravel'] = Tone_Ravel
data.head()


In [0]:
#привожу признаки к одной длине
#определяю максимальную длину
max_len = np.max(list(map(len,list(data['Tone.Ravel']))))
for i in range(len(data)):
  #создаю нулевой массив с недостающими нулями
  zero = np.zeros(max_len - len(data['Tone.Ravel'][i]), dtype=int)
  #добавляю нулевой массив в конец
  data['Tone.Ravel'][i] = np.hstack((data['Tone.Ravel'][i], zero))



In [0]:
#загружаю лейблы
labels = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv')
labels.head()

,fname,label
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping
1,00d77b917e241afa06f1.wav,Squeak
2,17bb93b73b8e79234cb3.wav,Electric_piano
3,7d5c7a40a936136da55e.wav,Harmonica
4,17e0ee7565a33d6c2326.wav,Snare_drum


In [0]:
#переименовываю название колонки для того чтобы потом объеденить признаки и лейблы в один дата фрейм
labels.rename(columns={'fname': 'Name_audio'}, inplace=True)

In [0]:
#объединяю 2 дата фрейма
all_data = data.merge(labels)
all_data.head()

,Name_audio,Audio,Feature_Tone,Tone.Ravel,label
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.028820622776791868, 0.035540443717563326, ...","[0.028820622776791868, 0.035540443717563326, 0...",Hi-hat
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.43014247714434317, 0.21300254030154597, 0....","[0.43014247714434317, 0.21300254030154597, 0.1...",Double_bass
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.26519665180603347, 0.2001198163939377, 0.1...","[0.26519665180603347, 0.2001198163939377, 0.17...",Applause
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.003857842635125953, 0.0006431823830839563,...","[0.003857842635125953, 0.0006431823830839563, ...",Oboe
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.12975261057595686, 0.3552813475665379, 0.0...","[0.12975261057595686, 0.3552813475665379, 0.01...",Flute


In [0]:
#получаю уникальные значения лейблов
unique_labels = np.unique(all_data['label'])
print(unique_labels )

['Acoustic_guitar' 'Applause' 'Bark' 'Bass_drum' 'Burping_or_eructation'
 'Bus' 'Cello' 'Chime' 'Clarinet' 'Computer_keyboard' 'Cough' 'Cowbell'
 'Double_bass' 'Drawer_open_or_close' 'Electric_piano' 'Fart'
 'Finger_snapping' 'Fireworks' 'Flute' 'Glockenspiel' 'Gong'
 'Gunshot_or_gunfire' 'Harmonica' 'Hi-hat' 'Keys_jangling' 'Knock'
 'Laughter' 'Meow' 'Microwave_oven' 'Oboe' 'Saxophone' 'Scissors'
 'Shatter' 'Snare_drum' 'Squeak' 'Tambourine' 'Tearing' 'Telephone'
 'Trumpet' 'Violin_or_fiddle' 'Writing']


In [0]:
for i in range(len(unique_labels)):
  print(unique_labels[i], len(list(all_data[all_data['label'] == unique_labels[i]]['label'])))

Acoustic_guitar 187
Applause 183
Bark 147
Bass_drum 181
Burping_or_eructation 113
Bus 64
Cello 186
Chime 78
Clarinet 176
Computer_keyboard 66
Cough 163
Cowbell 117
Double_bass 191
Drawer_open_or_close 94
Electric_piano 81
Fart 195
Finger_snapping 80
Fireworks 180
Flute 194
Glockenspiel 57
Gong 180
Gunshot_or_gunfire 88
Harmonica 85
Hi-hat 180
Keys_jangling 83
Knock 153
Laughter 174
Meow 99
Microwave_oven 90
Oboe 162
Saxophone 179
Scissors 56
Shatter 184
Snare_drum 173
Squeak 171
Tambourine 135
Tearing 162
Telephone 80
Trumpet 188
Violin_or_fiddle 167
Writing 161


In [0]:
#дата фрейм чтобы посмотреть сколько экземпляров в каждом классе попало в тест и трейн
Test_Train = pd.DataFrame (columns=['label','Test', 'Train', 'Sum'])
Test_Train.head()

,label,Test,Train,Sum


In [0]:
#разделяю на трейн и тест все данные в проморции 70 на 30 для каждого лейбла
X_train,  X_test,  Y_train,  Y_test = [], [], [], []
for i in range(len(unique_labels)):
  # получаю трейн и тест выборки для каждого лейбла
  X_train_one_label,  X_test_one_label,  Y_train_one_label,  Y_test_one_label =  train_test_split(list(all_data[all_data['label'] == unique_labels[i]]['Tone.Ravel']),  list(all_data[all_data['label'] == unique_labels[i]]['label']),  test_size = 0.3)
  Test_Train.loc[i] = [unique_labels[i], len(X_train_one_label), len(X_test_one_label),len(X_train_one_label) + len(X_test_one_label) ]
  # добавляю их в общий массив
  X_train.extend(X_train_one_label)
  X_test.extend(X_test_one_label)
  Y_train.extend(Y_train_one_label)
  Y_test.extend(Y_test_one_label)
  


In [0]:
Test_Train

,label,Test,Train,Sum
0,Acoustic_guitar,130,57,187
1,Applause,128,55,183
2,Bark,102,45,147
3,Bass_drum,126,55,181
4,Burping_or_eructation,79,34,113
5,Bus,44,20,64
6,Cello,130,56,186
7,Chime,54,24,78
8,Clarinet,123,53,176
9,Computer_keyboard,46,20,66


In [0]:
len(Y_train)

3960

In [0]:
len(Y_test)

1723

In [0]:
#наивный Байес
nb = GaussianNB()
nb.fit( X_train, Y_train)
Y_predict = nb.predict(X_test)

In [0]:
print(classification_report(Y_test,Y_predict, digits = 4))

                       precision    recall  f1-score   support

      Acoustic_guitar     0.0000    0.0000    0.0000        57
             Applause     0.0000    0.0000    0.0000        55
                 Bark     0.0000    0.0000    0.0000        45
            Bass_drum     0.0000    0.0000    0.0000        55
Burping_or_eructation     0.0000    0.0000    0.0000        34
                  Bus     0.0606    0.1000    0.0755        20
                Cello     0.0000    0.0000    0.0000        56
                Chime     0.0000    0.0000    0.0000        24
             Clarinet     0.0000    0.0000    0.0000        53
    Computer_keyboard     0.0185    0.0500    0.0270        20
                Cough     0.0323    0.0408    0.0360        49
              Cowbell     0.0000    0.0000    0.0000        36
          Double_bass     0.0432    0.9655    0.0827        58
 Drawer_open_or_close     0.0000    0.0000    0.0000        29
       Electric_piano     0.0000    0.0000    0.0000  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
#2 n_fft=1024 hop_length=512

In [0]:
#выгружаю из pickle признаки tone
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_1024_512.pickle', 'rb') as f:
...     Feature_Tone = pickle.load(f)

In [0]:
data['Feature_Tone'] = Feature_Tone
data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_1024_512.pickle', 'rb') as f:
...     Tone_Ravel = pickle.load(f)

In [56]:
data['Tone.Ravel'] = Tone_Ravel
data.head()


,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,NaN,"[[0.03591353273255355, 0.07999108756084874, 0....","[0.03591353273255355, 0.07999108756084874, 0.1..."
1,f9b80a8a5903c7006f7a.wav,NaN,"[[1.0, 0.2728053179609622, 0.19999485546241294...","[1.0, 0.2728053179609622, 0.19999485546241294,..."
2,70b9edbb2fb73de11b45.wav,NaN,"[[0.3212225773608423, 0.12028911571730426, 0.1...","[0.3212225773608423, 0.12028911571730426, 0.16..."
3,f98ff11b6ab64c31ac6d.wav,NaN,"[[0.21851610492290563, 0.0011322778021833017, ...","[0.21851610492290563, 0.0011322778021833017, 0..."
4,f98de3d40918553d4599.wav,NaN,"[[0.33004978266688545, 0.5684605529050966, 0.5...","[0.33004978266688545, 0.5684605529050966, 0.51..."


In [0]:
#привожу признаки к одной длине
#определяю максимальную длину
max_len = np.max(list(map(len,list(data['Tone.Ravel']))))
for i in range(len(data)):
  #создаю нулевой массив с недостающими нулями
  zero = np.zeros(max_len - len(data['Tone.Ravel'][i]), dtype=int)
  #добавляю нулевой массив в конец
  data['Tone.Ravel'][i] = np.hstack((data['Tone.Ravel'][i], zero))

In [0]:
#загружаю лейблы
labels = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv')
labels.head()

In [0]:
#переименовываю название колонки для того чтобы потом объеденить признаки и лейблы в один дата фрейм
labels.rename(columns={'fname': 'Name_audio'}, inplace=True)

In [61]:
#объединяю 2 дата фрейма
all_data = data.merge(labels)
all_data.head()

,Name_audio,Audio,Feature_Tone,Tone.Ravel,label
0,f9b737c1d2083b6a5a85.wav,NaN,"[[0.03591353273255355, 0.07999108756084874, 0....","[0.03591353273255355, 0.07999108756084874, 0.1...",Hi-hat
1,f9b80a8a5903c7006f7a.wav,NaN,"[[1.0, 0.2728053179609622, 0.19999485546241294...","[1.0, 0.2728053179609622, 0.19999485546241294,...",Double_bass
2,70b9edbb2fb73de11b45.wav,NaN,"[[0.3212225773608423, 0.12028911571730426, 0.1...","[0.3212225773608423, 0.12028911571730426, 0.16...",Applause
3,f98ff11b6ab64c31ac6d.wav,NaN,"[[0.21851610492290563, 0.0011322778021833017, ...","[0.21851610492290563, 0.0011322778021833017, 0...",Oboe
4,f98de3d40918553d4599.wav,NaN,"[[0.33004978266688545, 0.5684605529050966, 0.5...","[0.33004978266688545, 0.5684605529050966, 0.51...",Flute


In [0]:
#получаю уникальные значения лейблов
unique_labels = np.unique(all_data['label'])
#print(unique_labels )

In [0]:
#разделяю на трейн и тест все данные в проморции 70 на 30 для каждого лейбла
X_train,  X_test,  Y_train,  Y_test = [], [], [], []
for i in range(len(unique_labels)):
  # получаю трейн и тест выборки для каждого лейбла
  X_train_one_label,  X_test_one_label,  Y_train_one_label,  Y_test_one_label =  train_test_split(list(all_data[all_data['label'] == unique_labels[i]]['Tone.Ravel']),  list(all_data[all_data['label'] == unique_labels[i]]['label']),  test_size = 0.3)
  # добавляю их в общий массив
  X_train.extend(X_train_one_label)
  X_test.extend(X_test_one_label)
  Y_train.extend(Y_train_one_label)
  Y_test.extend(Y_test_one_label)
  


In [0]:
#наивный Байес
nb = GaussianNB()
nb.fit( X_train, Y_train)
Y_predict = nb.predict(X_test)

In [66]:
print(classification_report(Y_test,Y_predict, digits = 4))

                       precision    recall  f1-score   support

      Acoustic_guitar     0.0000    0.0000    0.0000        57
             Applause     0.0000    0.0000    0.0000        55
                 Bark     0.0000    0.0000    0.0000        45
            Bass_drum     0.0000    0.0000    0.0000        55
Burping_or_eructation     0.0000    0.0000    0.0000        34
                  Bus     0.1429    0.1500    0.1463        20
                Cello     0.0000    0.0000    0.0000        56
                Chime     0.0000    0.0000    0.0000        24
             Clarinet     0.0000    0.0000    0.0000        53
    Computer_keyboard     0.0000    0.0000    0.0000        20
                Cough     0.0000    0.0000    0.0000        49
              Cowbell     0.0000    0.0000    0.0000        36
          Double_bass     0.0404    1.0000    0.0777        58
 Drawer_open_or_close     0.0000    0.0000    0.0000        29
       Electric_piano     0.0000    0.0000    0.0000  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
#3 n_fft=2048 hop_length=1024

In [0]:
#выгружаю из pickle признаки tone
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_2048_1024.pickle', 'rb') as f:
...     Feature_Tone = pickle.load(f)

In [0]:
data['Feature_Tone'] = Feature_Tone
data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_2048_1024.pickle', 'rb') as f:
...     Tone_Ravel = pickle.load(f)

In [81]:
data['Tone.Ravel'] = Tone_Ravel
data.head()


,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.028820622776791868, 0.05756348137917222, 0...","[0.028820622776791868, 0.05756348137917222, 0...."
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.43014247714434317, 0.1747029704718087, 0.6...","[0.43014247714434317, 0.1747029704718087, 0.68..."
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.35261583312453576, 0.15936729123483687, 0....","[0.35261583312453576, 0.15936729123483687, 0.3..."
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.003857842635125953, 0.00031330631335511704...","[0.003857842635125953, 0.00031330631335511704,..."
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.12975261057595686, 0.013176895523852707, 0...","[0.12975261057595686, 0.013176895523852707, 0...."


In [0]:
#привожу признаки к одной длине
#определяю максимальную длину
max_len = np.max(list(map(len,list(data['Tone.Ravel']))))
for i in range(len(data)):
  #создаю нулевой массив с недостающими нулями
  zero = np.zeros(max_len - len(data['Tone.Ravel'][i]), dtype=int)
  #добавляю нулевой массив в конец
  data['Tone.Ravel'][i] = np.hstack((data['Tone.Ravel'][i], zero))

In [0]:
#загружаю лейблы
labels = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv')
labels.head()

In [0]:
#переименовываю название колонки для того чтобы потом объеденить признаки и лейблы в один дата фрейм
labels.rename(columns={'fname': 'Name_audio'}, inplace=True)

In [85]:
#объединяю 2 дата фрейма
all_data = data.merge(labels)
all_data.head()

,Name_audio,Audio,Feature_Tone,Tone.Ravel,label
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.028820622776791868, 0.05756348137917222, 0...","[0.028820622776791868, 0.05756348137917222, 0....",Hi-hat
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.43014247714434317, 0.1747029704718087, 0.6...","[0.43014247714434317, 0.1747029704718087, 0.68...",Double_bass
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.35261583312453576, 0.15936729123483687, 0....","[0.35261583312453576, 0.15936729123483687, 0.3...",Applause
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.003857842635125953, 0.00031330631335511704...","[0.003857842635125953, 0.00031330631335511704,...",Oboe
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.12975261057595686, 0.013176895523852707, 0...","[0.12975261057595686, 0.013176895523852707, 0....",Flute


In [0]:
#получаю уникальные значения лейблов
unique_labels = np.unique(all_data['label'])
#print(unique_labels )

In [0]:
#разделяю на трейн и тест все данные в проморции 70 на 30 для каждого лейбла
X_train,  X_test,  Y_train,  Y_test = [], [], [], []
for i in range(len(unique_labels)):
  # получаю трейн и тест выборки для каждого лейбла
  X_train_one_label,  X_test_one_label,  Y_train_one_label,  Y_test_one_label =  train_test_split(list(all_data[all_data['label'] == unique_labels[i]]['Tone.Ravel']),  list(all_data[all_data['label'] == unique_labels[i]]['label']),  test_size = 0.3)
  # добавляю их в общий массив
  X_train.extend(X_train_one_label)
  X_test.extend(X_test_one_label)
  Y_train.extend(Y_train_one_label)
  Y_test.extend(Y_test_one_label)
  


In [0]:
#наивный Байес
nb = GaussianNB()
nb.fit( X_train, Y_train)
Y_predict = nb.predict(X_test)

In [89]:
print(classification_report(Y_test,Y_predict, digits = 4))

                       precision    recall  f1-score   support

      Acoustic_guitar     0.0000    0.0000    0.0000        57
             Applause     0.0000    0.0000    0.0000        55
                 Bark     0.0000    0.0000    0.0000        45
            Bass_drum     0.0000    0.0000    0.0000        55
Burping_or_eructation     0.0000    0.0000    0.0000        34
                  Bus     0.1277    0.3000    0.1791        20
                Cello     0.0000    0.0000    0.0000        56
                Chime     0.0000    0.0000    0.0000        24
             Clarinet     0.0000    0.0000    0.0000        53
    Computer_keyboard     0.0000    0.0000    0.0000        20
                Cough     0.0000    0.0000    0.0000        49
              Cowbell     0.0000    0.0000    0.0000        36
          Double_bass     0.0186    0.1207    0.0322        58
 Drawer_open_or_close     0.0000    0.0000    0.0000        29
       Electric_piano     0.0000    0.0000    0.0000  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
#4 n_fft=2048 hop_length=2048

In [0]:
#выгружаю из pickle признаки tone
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_2048_2048.pickle', 'rb') as f:
...     Feature_Tone = pickle.load(f)

In [0]:
data['Feature_Tone'] = Feature_Tone
data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_2048_1024.pickle', 'rb') as f:
...     Tone_Ravel = pickle.load(f)

In [97]:
data['Tone.Ravel'] = Tone_Ravel
data.head()


,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.03544055988909569, 0.053576099300204884, 0...","[0.028820622776791868, 0.05756348137917222, 0...."
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.43014247714434317, 0.6819019300708192, 0.0...","[0.43014247714434317, 0.1747029704718087, 0.68..."
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.31337717351433225, 0.4140622522049719, 0.5...","[0.35261583312453576, 0.15936729123483687, 0.3..."
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.003857842635125953, 0.0002469295389883514,...","[0.003857842635125953, 0.00031330631335511704,..."
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.1316032193091004, 0.00020216236367174003, ...","[0.12975261057595686, 0.013176895523852707, 0...."


In [0]:
#привожу признаки к одной длине
#определяю максимальную длину
max_len = np.max(list(map(len,list(data['Tone.Ravel']))))
for i in range(len(data)):
  #создаю нулевой массив с недостающими нулями
  zero = np.zeros(max_len - len(data['Tone.Ravel'][i]), dtype=int)
  #добавляю нулевой массив в конец
  data['Tone.Ravel'][i] = np.hstack((data['Tone.Ravel'][i], zero))

In [0]:
#загружаю лейблы
labels = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv')
labels.head()

In [0]:
#переименовываю название колонки для того чтобы потом объеденить признаки и лейблы в один дата фрейм
labels.rename(columns={'fname': 'Name_audio'}, inplace=True)

In [101]:
#объединяю 2 дата фрейма
all_data = data.merge(labels)
all_data.head()

,Name_audio,Audio,Feature_Tone,Tone.Ravel,label
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.03544055988909569, 0.053576099300204884, 0...","[0.028820622776791868, 0.05756348137917222, 0....",Hi-hat
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.43014247714434317, 0.6819019300708192, 0.0...","[0.43014247714434317, 0.1747029704718087, 0.68...",Double_bass
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.31337717351433225, 0.4140622522049719, 0.5...","[0.35261583312453576, 0.15936729123483687, 0.3...",Applause
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.003857842635125953, 0.0002469295389883514,...","[0.003857842635125953, 0.00031330631335511704,...",Oboe
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.1316032193091004, 0.00020216236367174003, ...","[0.12975261057595686, 0.013176895523852707, 0....",Flute


In [0]:
#получаю уникальные значения лейблов
unique_labels = np.unique(all_data['label'])
#print(unique_labels )

In [0]:
#разделяю на трейн и тест все данные в проморции 70 на 30 для каждого лейбла
X_train,  X_test,  Y_train,  Y_test = [], [], [], []
for i in range(len(unique_labels)):
  # получаю трейн и тест выборки для каждого лейбла
  X_train_one_label,  X_test_one_label,  Y_train_one_label,  Y_test_one_label =  train_test_split(list(all_data[all_data['label'] == unique_labels[i]]['Tone.Ravel']),  list(all_data[all_data['label'] == unique_labels[i]]['label']),  test_size = 0.3)
  # добавляю их в общий массив
  X_train.extend(X_train_one_label)
  X_test.extend(X_test_one_label)
  Y_train.extend(Y_train_one_label)
  Y_test.extend(Y_test_one_label)
  


In [0]:
#наивный Байес
nb = GaussianNB()
nb.fit( X_train, Y_train)
Y_predict = nb.predict(X_test)

In [105]:
print(classification_report(Y_test,Y_predict, digits = 4))

                       precision    recall  f1-score   support

      Acoustic_guitar     0.0000    0.0000    0.0000        57
             Applause     0.0000    0.0000    0.0000        55
                 Bark     0.0000    0.0000    0.0000        45
            Bass_drum     0.0000    0.0000    0.0000        55
Burping_or_eructation     0.0000    0.0000    0.0000        34
                  Bus     0.3333    0.3000    0.3158        20
                Cello     0.0000    0.0000    0.0000        56
                Chime     0.0000    0.0000    0.0000        24
             Clarinet     0.0000    0.0000    0.0000        53
    Computer_keyboard     0.0000    0.0000    0.0000        20
                Cough     0.0000    0.0000    0.0000        49
              Cowbell     0.0000    0.0000    0.0000        36
          Double_bass     0.0433    0.9828    0.0830        58
 Drawer_open_or_close     0.0000    0.0000    0.0000        29
       Electric_piano     0.0000    0.0000    0.0000  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
#5 n_fft=2048 hop_length=4096

In [0]:
#выгружаю из pickle признаки tone
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_2048_4096.pickle', 'rb') as f:
...     Feature_Tone = pickle.load(f)

In [0]:
data['Feature_Tone'] = Feature_Tone
data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_2048_4096.pickle', 'rb') as f:
...     Tone_Ravel = pickle.load(f)

In [115]:
data['Tone.Ravel'] = Tone_Ravel
data.head()


,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.03544055988909569, 0.08681836079611299, 0....","[0.03544055988909569, 0.08681836079611299, 0.0..."
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.4229803343081988, 0.09249064997136945, 0.5...","[0.4229803343081988, 0.09249064997136945, 0.51..."
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.3405404610322879, 0.5165245503348904, 0.66...","[0.3405404610322879, 0.5165245503348904, 0.664..."
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.0038583998299812484, 0.0004918867050602356...","[0.0038583998299812484, 0.0004918867050602356,..."
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.1316032193091004, 5.8416944354149314e-05, ...","[0.1316032193091004, 5.8416944354149314e-05, 3..."


In [0]:
#привожу признаки к одной длине
#определяю максимальную длину
max_len = np.max(list(map(len,list(data['Tone.Ravel']))))
for i in range(len(data)):
  #создаю нулевой массив с недостающими нулями
  zero = np.zeros(max_len - len(data['Tone.Ravel'][i]), dtype=int)
  #добавляю нулевой массив в конец
  data['Tone.Ravel'][i] = np.hstack((data['Tone.Ravel'][i], zero))

In [0]:
#загружаю лейблы
labels = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv')
labels.head()

In [0]:
#переименовываю название колонки для того чтобы потом объеденить признаки и лейблы в один дата фрейм
labels.rename(columns={'fname': 'Name_audio'}, inplace=True)

In [119]:
#объединяю 2 дата фрейма
all_data = data.merge(labels)
all_data.head()

,Name_audio,Audio,Feature_Tone,Tone.Ravel,label
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.03544055988909569, 0.08681836079611299, 0....","[0.03544055988909569, 0.08681836079611299, 0.0...",Hi-hat
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.4229803343081988, 0.09249064997136945, 0.5...","[0.4229803343081988, 0.09249064997136945, 0.51...",Double_bass
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.3405404610322879, 0.5165245503348904, 0.66...","[0.3405404610322879, 0.5165245503348904, 0.664...",Applause
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.0038583998299812484, 0.0004918867050602356...","[0.0038583998299812484, 0.0004918867050602356,...",Oboe
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.1316032193091004, 5.8416944354149314e-05, ...","[0.1316032193091004, 5.8416944354149314e-05, 3...",Flute


In [0]:
#получаю уникальные значения лейблов
unique_labels = np.unique(all_data['label'])
#print(unique_labels )

In [0]:
#разделяю на трейн и тест все данные в проморции 70 на 30 для каждого лейбла
X_train,  X_test,  Y_train,  Y_test = [], [], [], []
for i in range(len(unique_labels)):
  # получаю трейн и тест выборки для каждого лейбла
  X_train_one_label,  X_test_one_label,  Y_train_one_label,  Y_test_one_label =  train_test_split(list(all_data[all_data['label'] == unique_labels[i]]['Tone.Ravel']),  list(all_data[all_data['label'] == unique_labels[i]]['label']),  test_size = 0.3)
  # добавляю их в общий массив
  X_train.extend(X_train_one_label)
  X_test.extend(X_test_one_label)
  Y_train.extend(Y_train_one_label)
  Y_test.extend(Y_test_one_label)
  


In [0]:
#наивный Байес
nb = GaussianNB()
nb.fit( X_train, Y_train)
Y_predict = nb.predict(X_test)

In [123]:
print(classification_report(Y_test,Y_predict, digits = 4))

                       precision    recall  f1-score   support

      Acoustic_guitar     0.0000    0.0000    0.0000        57
             Applause     0.0000    0.0000    0.0000        55
                 Bark     0.0000    0.0000    0.0000        45
            Bass_drum     0.0000    0.0000    0.0000        55
Burping_or_eructation     0.0000    0.0000    0.0000        34
                  Bus     0.1739    0.2000    0.1860        20
                Cello     0.0000    0.0000    0.0000        56
                Chime     0.0000    0.0000    0.0000        24
             Clarinet     0.0000    0.0000    0.0000        53
    Computer_keyboard     0.0283    0.1500    0.0476        20
                Cough     0.0000    0.0000    0.0000        49
              Cowbell     0.0000    0.0000    0.0000        36
          Double_bass     0.0416    0.9828    0.0799        58
 Drawer_open_or_close     0.0000    0.0000    0.0000        29
       Electric_piano     0.0000    0.0000    0.0000  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
#6 n_fft=4096 hop_length=4096

In [0]:
#выгружаю из pickle признаки tone
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_4096_4096.pickle', 'rb') as f:
...     Feature_Tone = pickle.load(f)

In [0]:
data['Feature_Tone'] = Feature_Tone
data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_4096_4096.pickle', 'rb') as f:
...     Tone_Ravel = pickle.load(f)

In [133]:
data['Tone.Ravel'] = Tone_Ravel
data.head()


,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.04185090192910369, 0.09652614059778075, 0....","[0.04185090192910369, 0.09652614059778075, 0.0..."
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.2002620515768468, 0.19608953641905738, 0.3...","[0.2002620515768468, 0.19608953641905738, 0.31..."
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.19268071089833572, 0.2915271758893053, 0.4...","[0.19268071089833572, 0.2915271758893053, 0.43..."
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.0006853054422468758, 0.0003786443135380064...","[0.0006853054422468758, 0.00037864431353800646..."
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.027898546237341153, 1.9090580513853288e-05...","[0.027898546237341153, 1.9090580513853288e-05,..."


In [0]:
#привожу признаки к одной длине
#определяю максимальную длину
max_len = np.max(list(map(len,list(data['Tone.Ravel']))))
for i in range(len(data)):
  #создаю нулевой массив с недостающими нулями
  zero = np.zeros(max_len - len(data['Tone.Ravel'][i]), dtype=int)
  #добавляю нулевой массив в конец
  data['Tone.Ravel'][i] = np.hstack((data['Tone.Ravel'][i], zero))

In [0]:
#загружаю лейблы
labels = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv')
labels.head()

In [0]:
#переименовываю название колонки для того чтобы потом объеденить признаки и лейблы в один дата фрейм
labels.rename(columns={'fname': 'Name_audio'}, inplace=True)

In [137]:
#объединяю 2 дата фрейма
all_data = data.merge(labels)
all_data.head()

,Name_audio,Audio,Feature_Tone,Tone.Ravel,label
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.04185090192910369, 0.09652614059778075, 0....","[0.04185090192910369, 0.09652614059778075, 0.0...",Hi-hat
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.2002620515768468, 0.19608953641905738, 0.3...","[0.2002620515768468, 0.19608953641905738, 0.31...",Double_bass
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.19268071089833572, 0.2915271758893053, 0.4...","[0.19268071089833572, 0.2915271758893053, 0.43...",Applause
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.0006853054422468758, 0.0003786443135380064...","[0.0006853054422468758, 0.00037864431353800646...",Oboe
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.027898546237341153, 1.9090580513853288e-05...","[0.027898546237341153, 1.9090580513853288e-05,...",Flute


In [0]:
#получаю уникальные значения лейблов
unique_labels = np.unique(all_data['label'])
#print(unique_labels )

In [0]:
#разделяю на трейн и тест все данные в проморции 70 на 30 для каждого лейбла
X_train,  X_test,  Y_train,  Y_test = [], [], [], []
for i in range(len(unique_labels)):
  # получаю трейн и тест выборки для каждого лейбла
  X_train_one_label,  X_test_one_label,  Y_train_one_label,  Y_test_one_label =  train_test_split(list(all_data[all_data['label'] == unique_labels[i]]['Tone.Ravel']),  list(all_data[all_data['label'] == unique_labels[i]]['label']),  test_size = 0.3)
  # добавляю их в общий массив
  X_train.extend(X_train_one_label)
  X_test.extend(X_test_one_label)
  Y_train.extend(Y_train_one_label)
  Y_test.extend(Y_test_one_label)
  


In [0]:
#наивный Байес
nb = GaussianNB()
nb.fit( X_train, Y_train)
Y_predict = nb.predict(X_test)

In [141]:
print(classification_report(Y_test,Y_predict, digits = 4))

                       precision    recall  f1-score   support

      Acoustic_guitar     0.0000    0.0000    0.0000        57
             Applause     0.0000    0.0000    0.0000        55
                 Bark     0.0000    0.0000    0.0000        45
            Bass_drum     0.0000    0.0000    0.0000        55
Burping_or_eructation     0.0000    0.0000    0.0000        34
                  Bus     0.1667    0.1500    0.1579        20
                Cello     0.0000    0.0000    0.0000        56
                Chime     0.0000    0.0000    0.0000        24
             Clarinet     0.0000    0.0000    0.0000        53
    Computer_keyboard     0.0000    0.0000    0.0000        20
                Cough     0.0000    0.0000    0.0000        49
              Cowbell     0.0000    0.0000    0.0000        36
          Double_bass     0.0072    0.0345    0.0119        58
 Drawer_open_or_close     0.0000    0.0000    0.0000        29
       Electric_piano     0.0000    0.0000    0.0000  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
#7 n_fft=4096 hop_length=2048

In [0]:
#выгружаю из pickle признаки tone
with open('/content/gdrive/My Drive/Colab Notebooks/Feature_Tone_4096_2048.pickle', 'rb') as f:
...     Feature_Tone = pickle.load(f)

In [0]:
data['Feature_Tone'] = Feature_Tone
data.head()

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Tone_Ravel_4096_2048.pickle', 'rb') as f:
...     Tone_Ravel = pickle.load(f)

In [149]:
data['Tone.Ravel'] = Tone_Ravel
data.head()


,Name_audio,Audio,Feature_Tone,Tone.Ravel
0,f9b737c1d2083b6a5a85.wav,"[-0.10846483, 0.05404515, 0.26760167, 0.026163...","[[0.032545723653486174, 0.050698541784126645, ...","[0.032545723653486174, 0.050698541784126645, 0..."
1,f9b80a8a5903c7006f7a.wav,"[-0.00081808947, -0.001746988, -0.002149351, -...","[[0.21944649740222488, 0.4776524497081728, 0.1...","[0.21944649740222488, 0.4776524497081728, 0.19..."
2,70b9edbb2fb73de11b45.wav,"[0.000449231, 0.0005544413, 0.00040870407, 9.5...","[[0.19268071089833572, 0.21827606310837194, 0....","[0.19268071089833572, 0.21827606310837194, 0.2..."
3,f98ff11b6ab64c31ac6d.wav,"[0.0002642143, 0.000118508644, -0.00014604167,...","[[0.0006904198605139633, 0.0002266265213019343...","[0.0006904198605139633, 0.0002266265213019343,..."
4,f98de3d40918553d4599.wav,"[-0.0018483361, -0.003198197, -0.0031196314, -...","[[0.018339949741691838, 0.00022532649119355767...","[0.018339949741691838, 0.00022532649119355767,..."


In [0]:
#привожу признаки к одной длине
#определяю максимальную длину
max_len = np.max(list(map(len,list(data['Tone.Ravel']))))
for i in range(len(data)):
  #создаю нулевой массив с недостающими нулями
  zero = np.zeros(max_len - len(data['Tone.Ravel'][i]), dtype=int)
  #добавляю нулевой массив в конец
  data['Tone.Ravel'][i] = np.hstack((data['Tone.Ravel'][i], zero))

In [0]:
#загружаю лейблы
labels = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv')
labels.head()

In [0]:
#переименовываю название колонки для того чтобы потом объеденить признаки и лейблы в один дата фрейм
labels.rename(columns={'fname': 'Name_audio'}, inplace=True)

In [0]:
#объединяю 2 дата фрейма
all_data = data.merge(labels)
all_data.head()

In [0]:
#получаю уникальные значения лейблов
unique_labels = np.unique(all_data['label'])
#print(unique_labels )

In [0]:
#разделяю на трейн и тест все данные в проморции 70 на 30 для каждого лейбла
X_train,  X_test,  Y_train,  Y_test = [], [], [], []
for i in range(len(unique_labels)):
  # получаю трейн и тест выборки для каждого лейбла
  X_train_one_label,  X_test_one_label,  Y_train_one_label,  Y_test_one_label =  train_test_split(list(all_data[all_data['label'] == unique_labels[i]]['Tone.Ravel']),  list(all_data[all_data['label'] == unique_labels[i]]['label']),  test_size = 0.3)
  # добавляю их в общий массив
  X_train.extend(X_train_one_label)
  X_test.extend(X_test_one_label)
  Y_train.extend(Y_train_one_label)
  Y_test.extend(Y_test_one_label)
  


In [0]:
#наивный Байес
nb = GaussianNB()
nb.fit( X_train, Y_train)
Y_predict = nb.predict(X_test)

In [157]:
print(classification_report(Y_test,Y_predict, digits = 4))

                       precision    recall  f1-score   support

      Acoustic_guitar     0.0000    0.0000    0.0000        57
             Applause     0.0000    0.0000    0.0000        55
                 Bark     0.0000    0.0000    0.0000        45
            Bass_drum     0.0000    0.0000    0.0000        55
Burping_or_eructation     0.0000    0.0000    0.0000        34
                  Bus     0.0870    0.1000    0.0930        20
                Cello     0.0000    0.0000    0.0000        56
                Chime     0.0000    0.0000    0.0000        24
             Clarinet     0.0000    0.0000    0.0000        53
    Computer_keyboard     0.0000    0.0000    0.0000        20
                Cough     0.0000    0.0000    0.0000        49
              Cowbell     0.0000    0.0000    0.0000        36
          Double_bass     0.0097    0.0517    0.0164        58
 Drawer_open_or_close     0.0000    0.0000    0.0000        29
       Electric_piano     0.0000    0.0000    0.0000  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
